In [1]:
import random
import warnings
warnings.filterwarnings("ignore")

# 加载需要的类库
from deap import base
from deap import creator
from deap import tools
import pandas as pd
import xgboost as xgb
import time
import numpy as np

# 定义投入的合金种类
set_future = {'氮化钒铁FeV55N11-A', '低铝硅铁', '钒氮合金(进口)',
          '钒铁(FeV50-B)', '硅铝钙', '硅铝合金FeAl30Si25',
          '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B', '石油焦增碳剂',
          '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)', '硅钙碳脱氧剂'}

# 定义 columns
columns = ['转炉终点温度', '转炉终点C', '转炉终点Mn', '转炉终点S', '转炉终点P', '转炉终点Si',
                                          '钢水净重', '氮化钒铁FeV55N11-A', '低铝硅铁', '钒氮合金(进口)',
                                          '钒铁(FeV50-B)', '硅铝钙', '硅铝合金FeAl30Si25',
                                          '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B', '石油焦增碳剂',
                                          '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)', '硅钙碳脱氧剂']

# 读取附件二中的个数据
read_path = "data/money_0.csv"
df_money_0 = pd.read_csv(read_path)
df_money_0 = df_money_0.fillna(0)

creator.create("FitnessMax", base.Fitness, weights=(-1.0,))  # 定义最大化适应度,确定优化目标为最小化
creator.create("Individual", list, fitness=creator.FitnessMax)  # 这里的list种群的数据类型

toolbox = base.Toolbox()

toolbox.register("attr_bool", random.random)  # 包含了0,1的随机整数,初始化种群

# Structure initializers: define 'individual' to be an individual
#                         consisting of 100 'attr_bool' elements ('genes')
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_bool, len(columns) - 7)

# define the population to be a list of 'individual's
toolbox.register("population", tools.initRepeat, list, toolbox.individual)



# 获取转化率
def Get_Yield(df_sample,element = "C"):

    element_CV_Model_new = xgb.Booster({'nthread': -1})  # init model
    element_CV_Model_new.load_model('model/'+ element +'_CV_Model.model')  # load data
    dmatrix_one = xgb.DMatrix(df_sample)
    return element_CV_Model_new.predict(dmatrix_one)

# 获取 连铸正样 中的各元素 比例
def Get_Content(df_sample,element="C",set_future = set_future):

    Yield = Get_Yield(df_sample=df_sample,element='C')
    
    All_element = 0
    for i in set(set_future):
        All_element += df_sample[i].iloc[0].values[0] * \
            df_money_0[df_money_0['合金配料'] == i][element].values[0]
    
    final_element = ((All_element * Yield +
                 df_sample['钢水净重'].iloc[0].values[0] * df_sample['转炉终点' + element].iloc[0].values[0])/df_sample['钢水净重'].iloc[0].values[0])
 
    
    return final_element[0]

# individual 转化为 pd.Dataframe
def individual2df(individual,list_init):
    list_sample = list(individual)
    
    list_init_2_df = list_init + [i* 1000 for i in list_sample]
    
    columns = ['转炉终点温度', '转炉终点C', '转炉终点Mn', '转炉终点S', '转炉终点P', '转炉终点Si',
                                              '钢水净重', '氮化钒铁FeV55N11-A', '低铝硅铁', '钒氮合金(进口)',
                                              '钒铁(FeV50-B)', '硅铝钙', '硅铝合金FeAl30Si25',
                                              '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B', '石油焦增碳剂',
                                              '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)', '硅钙碳脱氧剂']
    
    df_sample = pd.DataFrame([list_init_2_df])
    df_sample.columns = [columns]
    df_sample["锰硅合金FeMn64Si27(合格块)"]  = df_sample["锰硅合金FeMn64Si27(合格块)"] * 3
    df_sample["锰硅合金FeMn68Si18(合格块)"]  = df_sample["锰硅合金FeMn68Si18(合格块)"] * 3
    
    return df_sample

# 获取成本
def Get_Cost(df_sample,set_future = set_future):

    All_Cost = 0
    for i in set(set_future):
        All_Cost += df_sample[i].iloc[0].values[0] * \
            df_money_0[df_money_0['合金配料'] == i]["价格(元/吨)"].values[0] / 1000

    return All_Cost/(df_sample['钢水净重']/1000).iloc[0].values[0]

# list_one -> individual
# Get_Content(individual,element = "C",set_future=set_future)
Avg_Cost = 343.06933462317204
# Get_Cost(individual,,set_future=set_future)


# 评价函数
def evalOneMin(individual):
    
    list_init = [1644,0.00065,0.0011,0.0003,0.00014,0.004,74400]

    set_future = {'氮化钒铁FeV55N11-A', '低铝硅铁', '钒氮合金(进口)',
                  '钒铁(FeV50-B)', '硅铝钙', '硅铝合金FeAl30Si25',
                  '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B', '石油焦增碳剂',
                  '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)', '硅钙碳脱氧剂'}
    
    df_individual = individual2df(individual,list_init=list_init)

    HRB = {"C": 0.25, "Si": 0.80, "Mn": 1.60, "P": 0.045, "S": 0.045}

    C_sub_percentage = np.abs(
        (Get_Content(df_individual, element="C", set_future=set_future) - HRB["C"]/100)/(HRB["C"]/100))
    Mn_sub_percentage = np.abs((Get_Content(
        df_individual, element="Mn", set_future=set_future) - HRB["Mn"]/100)/(HRB["Mn"]/100))
    Si_sub_percentage = np.abs((Get_Content(
        df_individual, element="Si", set_future=set_future) - HRB["Si"]/100)/(HRB["Si"]/100))

    All_Cost_sub_percentage = (Get_Cost(df_individual, set_future=set_future) - Avg_Cost)/Avg_Cost

    sums =  All_Cost_sub_percentage + 10 * C_sub_percentage + 10 * Mn_sub_percentage + 10 * Si_sub_percentage

    return sums,


# ---------------------Operator registration---------------------
toolbox.register("evaluate", evalOneMin)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

def showData(best_ind,list_init = [1644,0.00065,0.0011,0.0003,0.00014,0.004,74400],show_list = False):
    set_future = {'氮化钒铁FeV55N11-A', '低铝硅铁', '钒氮合金(进口)',
              '钒铁(FeV50-B)', '硅铝钙', '硅铝合金FeAl30Si25',
              '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B', '石油焦增碳剂',
              '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)', '硅钙碳脱氧剂'}
    
    if len(best_ind) == 21:
        best_ind = best_ind[7:]        
    df_individual = individual2df(best_ind,list_init=list_init)
    
    All_Cost_sub_percentage = (Get_Cost(df_individual, set_future=set_future) - Avg_Cost)/Avg_Cost
    
    if show_list:
        print("最终结果为 :",list(df_individual.values))    
    
    HRB = {"C": 0.25, "Si": 0.80, "Mn": 1.60, "P": 0.045, "S": 0.045}


    C_sub_percentage = np.abs(
        (Get_Content(df_individual, element="C", set_future=set_future) - HRB["C"]/100)/(HRB["C"]/100))
    Mn_sub_percentage = np.abs((Get_Content(
        df_individual, element="Mn", set_future=set_future) - HRB["Mn"]/100)/(HRB["Mn"]/100))
    Si_sub_percentage = np.abs((Get_Content(
        df_individual, element="Si", set_future=set_future) - HRB["Si"]/100)/(HRB["Si"]/100))

    print(All_Cost_sub_percentage,C_sub_percentage,Mn_sub_percentage,Si_sub_percentage)
# --------------------- main ---------------------

def main(NGEN = 100):
    random.seed(42)

    pop = toolbox.population(n=300)  # 定义300个个体的种群

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    #
    # NGEN  is the number of generations for which the
    #       evolution runs   迭代次数
    CXPB, MUTPB = 0.5, 0.2

    print("Start of evolution")

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))

    print("  Evaluated %i individuals" % len(pop))  # 这时候，pop的长度还是100呢
    print("  迭代 %i 次" % NGEN)

    # Begin the evolution      开始进化了哈！！！注意注意注意！就是一个for循环里了！NGEN次--代数
    for g in range(NGEN):
        if g % 10 == 0:
            print("-- Generation %i --" % (g+1))
            print("-- 开始第 %d 次运行,现在的时间是 :"%(g+1),time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))

        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        pop[:] = offspring

        best_ind = tools.selBest(pop, 1)[0]

        showData(best_ind)

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    
    return best_ind

d:\anconda\envs\dl\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


In [2]:
'''
最终结果为 : [array([1.64400000e+03, 6.50000000e-04, 1.10000000e-03, 3.00000000e-04,
       1.40000000e-04, 4.00000000e-03, 7.44000000e+04, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.25637515e+01,
       1.01752049e+02, 8.71979330e+02, 0.00000000e+00, 0.00000000e+00,
       1.12937479e+02, 1.10268315e+03, 0.00000000e+00, 0.00000000e+00,
       1.45328436e+02])]
-0.3122357585236908 0.05943478820144479 0.003737911101310467 0.013063820459509367

 -- 样本一,最终结果 -- 
最终结果为 : [array([1.64400000e+03, 6.50000000e-04, 1.10000000e-03, 3.00000000e-04,
       1.40000000e-04, 4.00000000e-03, 7.44000000e+04, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.25637515e+01,
       1.01752049e+02, 8.71979330e+02, 0.00000000e+00, 0.00000000e+00,
       1.12937479e+02, 1.10268315e+03, 0.00000000e+00, 0.00000000e+00,
       1.45328436e+02])]
-0.31223575852369095 0.05943478820144479 0.003737911101310467 0.013063820459509367
'''

best_ind = main(NGEN = 50)
list_init = [1644,0.00065,0.0011,0.0003,0.00014,0.004,74400]
print(' -- 样本一,最终结果 -- ')
showData(best_ind,show_list = True)

Start of evolution
  Evaluated 300 individuals
  迭代 50 次
-- Generation 1 --
-- 开始第 1 次运行,现在的时间是 : 2019-04-14 12:43:10
14.800315617188382 0.11327616373697919 0.5229588170205394 1.3596224220850133
14.370496877311753 0.13582356770833331 0.1452032519925025 1.0724920047226771
6.27707415181712 0.11827513992145496 0.5229588170205394 1.2788275441815775
7.559631503933617 0.2421052173901629 0.07135188195013238 0.910645269578503
2.6219460688544625 0.2418666552471858 0.07135188195013238 0.8153248038343204
7.353891063304376 0.1287886096585181 0.00589823979203416 0.20471359581075688
4.868375268731624 0.1287886096585181 0.00589823979203416 0.20471359581075688
2.477422254645956 0.1287886096585181 0.00589823979203416 0.37410727880334327
2.477422254645956 0.1287886096585181 0.00589823979203416 0.37410727880334327
2.1847668219165532 0.1308434599189348 0.003737911101310467 0.10749501054004951
-- Generation 11 --
-- 开始第 11 次运行,现在的时间是 : 2019-04-14 12:47:19
-0.04887349226387069 0.1287886096585181 0.005898239

In [3]:
'''
 -- 样本二,最终结果 -- 
最终结果为 : [array([1.65100000e+03, 4.10000000e-04, 1.10000000e-03, 3.70000000e-04,
       2.80000000e-04, 3.00000000e-03, 7.00000000e+04, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.25637515e+01,
       1.01752049e+02, 8.71979330e+02, 0.00000000e+00, 0.00000000e+00,
       1.12937479e+02, 1.10268315e+03, 0.00000000e+00, 0.00000000e+00,
       1.45328436e+02])]
-0.2690048633451801 0.002456839425223241 0.04937656947544646 0.0868924277169364

'''

def evalOneMin(individual):
    
    list_init = [1651,0.00041,0.0011,0.00037,0.00028,0.003,70000]

    set_future = {'氮化钒铁FeV55N11-A', '低铝硅铁', '钒氮合金(进口)',
                  '钒铁(FeV50-B)', '硅铝钙', '硅铝合金FeAl30Si25',
                  '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B', '石油焦增碳剂',
                  '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)', '硅钙碳脱氧剂'}

    df_individual = individual2df(individual,list_init=list_init)

    HRB = {"C": 0.25, "Si": 0.80, "Mn": 1.60, "P": 0.045, "S": 0.045}

    C_sub_percentage = np.abs(
        (Get_Content(df_individual, element="C", set_future=set_future) - HRB["C"]/100)/(HRB["C"]/100))
    Mn_sub_percentage = np.abs((Get_Content(
        df_individual, element="Mn", set_future=set_future) - HRB["Mn"]/100)/(HRB["Mn"]/100))
    Si_sub_percentage = np.abs((Get_Content(
        df_individual, element="Si", set_future=set_future) - HRB["Si"]/100)/(HRB["Si"]/100))

    All_Cost_sub_percentage = (Get_Cost(df_individual, set_future=set_future) - Avg_Cost)/Avg_Cost

    sums =  All_Cost_sub_percentage + 10 * C_sub_percentage + 10 * Mn_sub_percentage + 10 * Si_sub_percentage

    return sums,

best_ind = main(NGEN = 50)
list_init_3 = [1651,0.00041,0.0011,0.00037,0.00028,0.003,70000]
print(' -- 样本二,最终结果 -- ')
showData(best_ind,list_init=list_init_3,show_list = True)

Start of evolution
  Evaluated 300 individuals
  迭代 50 次
-- Generation 1 --
-- 开始第 1 次运行,现在的时间是 : 2019-04-14 13:03:02
14.800315617188382 0.11327616373697919 0.5229588170205394 1.3596224220850133
14.370496877311753 0.13582356770833331 0.1452032519925025 1.0724920047226771
6.27707415181712 0.11827513992145496 0.5229588170205394 1.2788275441815775
7.559631503933617 0.2421052173901629 0.07135188195013238 0.910645269578503
2.6219460688544625 0.2418666552471858 0.07135188195013238 0.8153248038343204
7.353891063304376 0.1287886096585181 0.00589823979203416 0.20471359581075688
4.868375268731624 0.1287886096585181 0.00589823979203416 0.20471359581075688
2.477422254645956 0.1287886096585181 0.00589823979203416 0.37410727880334327
2.477422254645956 0.1287886096585181 0.00589823979203416 0.37410727880334327
2.1847668219165532 0.1308434599189348 0.003737911101310467 0.10749501054004951
-- Generation 11 --
-- 开始第 11 次运行,现在的时间是 : 2019-04-14 13:07:09
-0.04887349226387069 0.1287886096585181 0.005898239

In [4]:
'''
 -- 样本三,最终结果 -- 
最终结果为 : [array([1.68400000e+03, 3.50000000e-04, 1.10000000e-03, 2.40000000e-04,
       2.00000000e-04, 4.00000000e-03, 7.82500000e+04, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.25637515e+01,
       1.01752049e+02, 8.71979330e+02, 0.00000000e+00, 0.00000000e+00,
       1.12937479e+02, 1.10268315e+03, 0.00000000e+00, 0.00000000e+00,
       1.45328436e+02])]
-0.3460746381362633 0.05649203461960857 0.008501729264426792 0.015678040135782806
'''

def evalOneMin(individual):
    
    list_init = [1684,0.00035,0.0011,0.00024,0.0002,0.004,78250]

    set_future = {'氮化钒铁FeV55N11-A', '低铝硅铁', '钒氮合金(进口)',
                  '钒铁(FeV50-B)', '硅铝钙', '硅铝合金FeAl30Si25',
                  '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B', '石油焦增碳剂',
                  '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)', '硅钙碳脱氧剂'}

    df_individual = individual2df(individual,list_init=list_init)

    HRB = {"C": 0.25, "Si": 0.80, "Mn": 1.60, "P": 0.045, "S": 0.045}

    C_sub_percentage = np.abs(
        (Get_Content(df_individual, element="C", set_future=set_future) - HRB["C"]/100)/(HRB["C"]/100))
    Mn_sub_percentage = np.abs((Get_Content(
        df_individual, element="Mn", set_future=set_future) - HRB["Mn"]/100)/(HRB["Mn"]/100))
    Si_sub_percentage = np.abs((Get_Content(
        df_individual, element="Si", set_future=set_future) - HRB["Si"]/100)/(HRB["Si"]/100))

    All_Cost_sub_percentage = (Get_Cost(df_individual, set_future=set_future) - Avg_Cost)/Avg_Cost

    sums =  All_Cost_sub_percentage + 10 * C_sub_percentage + 10 * Mn_sub_percentage + 10 * Si_sub_percentage

    return sums,

best_ind = main(NGEN = 50)
list_init_3 = [1684,0.00035,0.0011,0.00024,0.0002,0.004,78250]
print(' -- 样本三,最终结果 -- ')
showData(best_ind,list_init=list_init_3,show_list = True)

Start of evolution
  Evaluated 300 individuals
  迭代 50 次
-- Generation 1 --
-- 开始第 1 次运行,现在的时间是 : 2019-04-14 13:22:45
14.800315617188382 0.11327616373697919 0.5229588170205394 1.3596224220850133
14.370496877311753 0.13582356770833331 0.1452032519925025 1.0724920047226771
6.27707415181712 0.11827513992145496 0.5229588170205394 1.2788275441815775
7.559631503933617 0.2421052173901629 0.07135188195013238 0.910645269578503
2.6219460688544625 0.2418666552471858 0.07135188195013238 0.8153248038343204
7.353891063304376 0.1287886096585181 0.00589823979203416 0.20471359581075688
4.868375268731624 0.1287886096585181 0.00589823979203416 0.20471359581075688
2.477422254645956 0.1287886096585181 0.00589823979203416 0.37410727880334327
2.477422254645956 0.1287886096585181 0.00589823979203416 0.37410727880334327
2.1847668219165532 0.1308434599189348 0.003737911101310467 0.10749501054004951
-- Generation 11 --
-- 开始第 11 次运行,现在的时间是 : 2019-04-14 13:26:52
-0.04887349226387069 0.1287886096585181 0.005898239

In [ ]:
# 评价函数
def evalOneMin(individual):
    
    list_init = [1684,0.00035,0.0011,0.00024,0.0002,0.004,78250]

    set_future = {'氮化钒铁FeV55N11-A', '低铝硅铁', '钒氮合金(进口)',
                  '钒铁(FeV50-B)', '硅铝钙', '硅铝合金FeAl30Si25',
                  '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B', '石油焦增碳剂',
                  '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)', '硅钙碳脱氧剂'}

    df_individual = individual2df(individual,list_init=list_init)

    HRB = {"C": 0.25, "Si": 0.80, "Mn": 1.60, "P": 0.045, "S": 0.045}

    C_sub_percentage = np.abs(
        (Get_Content(df_individual, element="C", set_future=set_future) - HRB["C"])/HRB["C"])
    Mn_sub_percentage = np.abs((Get_Content(
        df_individual, element="Mn", set_future=set_future) - HRB["Mn"])/HRB["Mn"])
    Si_sub_percentage = np.abs((Get_Content(
        df_individual, element="Si", set_future=set_future) - HRB["Si"])/HRB["Si"])

    All_Cost_sub_percentage = (Get_Cost(df_individual, set_future=set_future) - Avg_Cost)/Avg_Cost

    sums =  3 * All_Cost_sub_percentage + C_sub_percentage + Mn_sub_percentage + Si_sub_percentage

    return sums,


def main():
    random.seed(42)
    pop = toolbox.population(n=300)  # 定义300个个体的种群
    CXPB, MUTPB, NGEN = 0.5, 0.2, 1000
    print("Start of evolution")
    fitnesses = list(map(toolbox.evaluate, pop))

    print("  Evaluated %i individuals" % len(pop))  # 这时候，pop的长度还是100呢
    print("  迭代 %i 次" % NGEN)

    # Begin the evolution      开始进化了哈！！！注意注意注意！就是一个for循环里了！NGEN次--代数
    for g in range(NGEN):
        if g % 10 == 0:
            print("-- Generation %i --" % (g+1))
            print("-- 开始第 %d 次运行,现在的时间是 :"%(g+1),time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))

        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))

        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        pop[:] = offspring

        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x * x for x in fits)
        std = abs(sum2 / length - mean ** 2) ** 0.5


    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    return best_ind


best_ind = main()

list_init_3 = [1684,0.00035,0.0011,0.00024,0.0002,0.004,78250]

df_individual = individual2df(best_ind,list_init=list_init_3)
All_Cost_sub_percentage = (Get_Cost(df_individual, set_future=set_future) - Avg_Cost)/Avg_Cost
print("成本的百分比变化为 : %s",All_Cost_sub_percentage)


C_sub_percentage = np.abs(
    (Get_Content(df_individual, element="C", set_future=set_future) - HRB["C"])/HRB["C"])
Mn_sub_percentage = np.abs((Get_Content(
    df_individual, element="Mn", set_future=set_future) - HRB["Mn"])/HRB["Mn"])
Si_sub_percentage = np.abs((Get_Content(
    df_individual, element="Si", set_future=set_future) - HRB["Si"])/HRB["Si"])

C_sub_percentage,Mn_sub_percentage,Si_sub_percentage